In [94]:
import pandas as pd
import numpy as np
import re
from re import search

## Some cleaning functions

In [95]:
def clean_website(s):

    if search("projetocomprova", s):  
        s = s.split("/")
        s = s[2]
    elif search("folha", s):
        s = s.strip("https://w{3,}\d{1,}.")
        s = s.split("/")
        s = s[0]
    else:
        s = s.strip("https://")
        s = s.strip("*w{3,}+.")
        s = s.split("/")
        s = s[0]
    
    return s

In [96]:
def clean_special_char(f):
    
    special = '\W'
    f = re.sub("#boato", "", f)
    f = re.sub(special, " ", f)
    
    return f

In [97]:
def clean_date(d):
    
    if re.search("Atualizada em", d) != None:
        d = d.split(" ")
        d = d[-3]
    elif re.search("Publicado em", d)!= None:
        d = d.split(" ")
        d = d[5]
    elif re.search(" às", d)!= None:
        d = d.split(" ")
        d = d[0]
    elif re.search("\d{1}\W\d{2}\W\d{5}",d) !=None:
        d = d[0:9]
    
    else:
        pass
    
    return d

In [98]:
def clean_author(a):

    if re.search("\d{2}\W\d{2}\W\d{4}\s\d{2}\w{3}", a) != None:
        a = "NaN"
    elif re.search('\w{100,}',a) != None:
        a = "NaN"
    elif re.search("[Pp]or",a) != None:
        a = re.sub("[Pp]or", "", a) 
    else:
        pass

    special = "\W"
    a = re.sub(special, "", a)
    
    return a

### Cleaning FakeRecogna
- Text column is already tokenized and lemmatized; #ok
- Lower characters of every row;
- There are special characters in the title; #ok
- Standard date; #ok
- Date and author are mixed up; almost #ok
- Excract news agency from URL; #ok
- There is a column of subtitles; #ok 
- Columns names are in Portuguese, translate to EN lower; #ok 
- Where the is date instead of author, input None; #ok
- Rating 0 for fake, 1 for real, as float. #ok

In [100]:
fr = pd.read_excel("C:\\Users\\franc\\IH-Lab\\BR-FakeNews-Detector\\Data\\FakeRecogna.xlsx")

In [101]:
#Columns names are in Portuguese, translate to EN lower;

In [102]:
fr.columns = ['title', "subtitle", "text", "tag", "date", "author", "url", "rating"]

In [103]:
#drop duplicates if any

In [104]:
fr.drop_duplicates(subset=['text'], inplace=True)

In [105]:
#lower characters of each row

for column in fr.columns:
    fr[column] = fr[column].str.lower()

In [106]:
#There are special characters in the title or text;

In [107]:
fr['title_processed'] = fr['title'].astype(str).apply(clean_special_char)

In [108]:
fr['text_processed'] = fr['text'].astype(str).apply(clean_special_char)

In [109]:
#return only agency name instead of url

In [110]:
fr['url_processed'] = fr['url'].astype(str).apply(clean_website)

In [111]:
# return standard date

In [112]:
fr['date_processed'] = fr['date'].astype(str).apply(clean_date)

In [113]:
fr['date_processed'] = pd.to_datetime(fr['date_processed'], errors='coerce').dt.date

In [114]:
# Return Nan when there is a date instead of author

In [115]:
fr['author_processed'] = fr['author'].astype(str).apply(clean_author)

In [116]:
fr['author_processed'] = np.where(fr['author_processed'] == "NaN", fr["url_processed"], fr['author_processed'])

In [117]:
#Dealing with Nans

In [118]:
np.where(fr['date_processed'].isna())

(array([    5,   106,   166,   189,   192,   206,   220,   284,   327,
          333,   348,   359,   382,   445,   452,   473,   498,   550,
          582,   611,   628,   662,   672,   771,   810,   813,   880,
          887,   911,   926,  1011,  1036,  1180,  1187,  1253,  1254,
         1387,  1421,  1482,  1569,  1604,  1630,  1631,  1643,  1659,
         1664,  1676,  1740,  1745,  1763,  1798,  1842,  1865,  1867,
         1875,  1923,  2038,  2045,  2056,  2068,  2077,  2094,  2183,
         2185,  2199,  2201,  2202,  2231,  2357,  2358,  2382,  2420,
         2439,  2481,  2504,  2508,  2620,  2632,  2640,  2701,  2703,
         2766,  2773,  2777,  2856,  2864,  2894,  2910,  2932,  2969,
         3017,  3102,  3112,  3140,  3160,  3165,  3221,  3267,  3276,
         3315,  3362,  3377,  3386,  3451,  3452,  3464,  3494,  3553,
         3657,  3670,  3681,  3697,  3722,  3734,  3786,  3809,  3919,
         3934,  3982,  4014,  4068,  4145,  4174,  4192,  4239,  4247,
      

In [119]:
dt = list(np.where(fr['date_processed'].isna()))

In [120]:
fr['date'][359]

nan

In [ ]:
fr.isna().sum() 

In [122]:
fr = fr.dropna(subset = ['text'])

In [123]:
it = list(np.where(fr['title_processed'].isna()))

In [124]:
for i in it:
    fr['title_processed'].iloc[i] = fr['text_processed'].iloc[i][0:50]

In [125]:
fr.isin(["NaN"]).sum() 

title               0
subtitle            0
text                0
tag                 0
date                0
author              0
url                 0
rating              0
title_processed     0
text_processed      0
url_processed       0
date_processed      0
author_processed    0
dtype: int64

In [126]:
fr['rating'].value_counts()

1.0    5951
0.0    5935
Name: rating, dtype: int64

In [127]:
fr['tag'].value_counts()

saúde             4454
política          3941
entretenimento    1405
brasil             904
ciência            602
mundo              580
Name: tag, dtype: int64

In [128]:
fr = fr.drop(['title', 'text', 'url', 'date', 'author'], axis = 1)

In [129]:
fr.columns

Index(['subtitle', 'tag', 'rating', 'title_processed', 'text_processed',
       'url_processed', 'date_processed', 'author_processed'],
      dtype='object')

In [130]:
fr.columns = ["subtitle", "tag", "rating", "title", "text", "url", "date", "author"]

In [131]:
fr = fr.reindex(['title', "subtitle", "text", "tag", "date", "author", "url", "rating"], axis=1)

In [ ]:
fr.to_csv("fr_cleaned_pro.tsv")

### Cleaning AFP Checamos
- Tokenize and lemmatize data;
- Lower characters of Text's every row; #ok
- Remove special characters from text; #ok
- Remove hour; #ok
- Include subtitles column with None; #ok
- Convert tags to fr's; #ok
- Excract news agency from URL; #ok
- Since authors were not provided, call them AFP Brasil; #ok
- Convert Ratings into 0 for fake, 1 for real; #ok
- Convert date to datetime. #ok

In [152]:
afp = pd.read_csv("C:\\Users\\franc\\IH-Lab\\BR-FakeNews-Detector\\Data\\afp.tsv", sep='\t').drop(["Unnamed: 0", "hour"], axis = 1)

In [153]:
afp['subtitle'] = None

In [154]:
afp['url'] = afp['url'].astype(str).apply(clean_website) 

In [155]:
afp['text'] = afp['text'].astype(str).apply(clean_special_char)

In [156]:
#Classify according to fr's tags: saúde, política, entretenimento, brasil, ciência, mundo,

In [157]:
afp['tag'].value_counts()

Conflito na Ucrânia     50
Nan                     42
Eleições 2022           27
COVID-19                 9
VACINAS                  4
Clima                    2
Eleições França 2022     1
Name: tag, dtype: int64

In [158]:
def clean_tag(t):
    
    if t == "Conflito na Ucrânia" or t == "Eleições França 2022":
        t = "mundo"
    elif t == "COVID-19" or t == "VACINAS":
        t = "saúde"
    elif t == "Clima":
        t = "ciência"
    elif t == "Eleições 2022":
        t = "brasil"
    elif t == "Nan":
        t = "política"
    else:
        pass
    
    return t

In [159]:
afp['tag'] = afp['tag'].astype(str).apply(clean_tag)

In [160]:
afp['author'] = "AFP Brasil"

In [161]:
afp['rating'].value_counts()

Falso           83
Enganoso        27
Sem contexto    11
Montagem         9
Sem registro     2
Sem Registro     1
Sem Contexto     1
Nan              1
Name: rating, dtype: int64

In [162]:
afp['rating']= 0.0

In [163]:
afp['date'] = pd.to_datetime(afp['date'], errors='coerce').dt.date

In [164]:
afp['text'] = afp['text'].apply(str.lower)

In [165]:
for column in afp.columns:
    afp[column] = np.where(afp[column].isin(["Nan"]), None, afp[column])

In [166]:
afp = afp.reindex(['title', "subtitle", "text", "tag", "date", "author", "url", "rating"], axis=1)

#### Token and lemmatize

In [1610]:
afp

,title,subtitle,text,tag,date,author,url,rating,text_processed
0,Vídeo em que cantora parece e...,None,um vídeo em que a cantora teresa cristina par...,brasil,2022-05-16,AFP Brasil,checamos.afp.com,0.0,"[víd, cantor, teres, cristin, parec, comet, di..."
1,Os 62 °C na Índia foram regis...,None,um mapa da índia foi compartilhado nas redes ...,ciência,2022-05-14,AFP Brasil,checamos.afp.com,0.0,"[map, índi, compartilh, red, soc, desd, 29, ab..."
2,"Nesta gravação de 2017, Lula ...",None,lula recusa descer do transporte para falar ...,brasil,2022-05-13,AFP Brasil,checamos.afp.com,0.0,"[lul, recus, desc, transport, fal, pesso, rua,..."
3,"É uma montagem a ""notícia"" qu...",None,a captura de tela de uma suposta notícia publ...,brasil,2022-05-13,AFP Brasil,checamos.afp.com,0.0,"[captur, tel, supost, notíc, public, pel, folh..."
4,Tanques em vídeo viral eram l...,None,uma gravação visualizada mais de 68 mil vezes...,mundo,2022-05-13,AFP Brasil,checamos.afp.com,0.0,"[gravaçã, visualiz, 68, mil, vez, red, soc, de..."
...,...,...,...,...,...,...,...,...,...
130,Foto de criança chorando não ...,None,a foto de uma criança chorando em meio a um c...,mundo,2022-01-03,AFP Brasil,checamos.afp.com,0.0,"[fot, crianc, chor, cenári, guerr, compartilh,..."
131,A foto do presidente ucranian...,None,uma fotografia do presidente ucraniano volod...,mundo,2022-02-25,AFP Brasil,checamos.afp.com,0.0,"[fotograf, president, ucranian, volodim, zelen..."
132,Vídeo de avião em chamas foi ...,None,a gravação de um avião que irrompe em chamas ...,mundo,2022-02-25,AFP Brasil,checamos.afp.com,0.0,"[gravaçã, aviã, irromp, cham, plen, voo, explo..."
133,Vídeo de aviões em formação é...,None,em meio à invasão da rússia à ucrânia em 24 d...,mundo,2022-02-25,AFP Brasil,checamos.afp.com,0.0,"[invasã, rúss, ucrân, 24, fevereir, 2022, cent..."


In [167]:
afp.to_csv("afp_cleaned_pro.tsv")

### Cleaning AosFatos:
- Tokenize and lemmatize data;
- Remove special characters; #ok
- Lower characters of Text's every row; #ok
- Remove hour; #ok
- Include subtitles column with None; #ok
- Excract news agency from URL; #ok
- Convert Ratings into 0 for fake, 1 for real; #ok
- Convert date to datetime. #ok

In [168]:
aosfatos = pd.read_csv("C:\\Users\\franc\\IH-Lab\\BR-FakeNews-Detector\\Data\\aosfatos.tsv", sep='\t').drop(["Unnamed: 0", "hour"], axis = 1)

In [169]:
aosfatos

,url,title,author,date,text,rating
0,https://www.aosfatos.org//noticias/video-mar-a...,Vídeo em que mar invade avenida no Rio de Jane...,Priscila Pacheco,2022-05-23,Não foi gravado recentemente um vídeo que most...,distorcido
1,https://www.aosfatos.org//noticias/aviao-lula-xp/,Avião usado por Lula em 2019 não pertencia à X...,Priscila Pacheco,2022-05-23,É falso que a XP Investimentos emprestou um av...,falso
2,https://www.aosfatos.org//noticias/lula-capa-f...,Lula não apareceu em capa da ‘Forbes’ como uma...,Marco Faustino,2022-05-20,É falso que uma capa da revista Forbes indica ...,falso
3,https://www.aosfatos.org//noticias/eduardo-cam...,É falso que Eduardo Campos liderava pesquisas ...,Priscila Pacheco,2022-05-20,O ex-governador de Pernambuco Eduardo Campos (...,falso
4,https://www.aosfatos.org//noticias/curitiba-gr...,Vídeo mostra via em Curitiba coberta de graniz...,Marco Faustino,2022-05-19,Um vídeo que mostra uma via coberta de gelo em...,falso
...,...,...,...,...,...,...
235,https://www.aosfatos.org//noticias/e-falso-que...,É falso que Estados Unidos baniram testes RT-P...,Marco Faustino,2022-01-06,Não é verdade que os Estados Unidos baniram os...,falso
236,https://www.aosfatos.org//noticias/e-falso-que...,É falso que vídeo mostre que barragem tem risc...,Priscila Pacheco,2022-01-05,Não é verdade que vídeo compartilhado nas rede...,falso
237,https://www.aosfatos.org//noticias/nao-e-verda...,Não é verdade que Ivete Sangalo anunciou que d...,Marco Faustino,2022-01-05,É falso que a cantora Ivete Sangalo anunciou q...,falso
238,https://www.aosfatos.org//noticias/video-que-m...,Vídeo que mostra protesto contra Lula não é re...,Marco Faustino,2022-01-04,Não é recente nem foi gravado no Recife (PE) o...,falso


In [170]:
aosfatos['url'] = aosfatos['url'].astype(str).apply(clean_website)

In [171]:
aosfatos['text'] = aosfatos['text'].astype(str).apply(clean_special_char)

In [172]:
aosfatos['title'] = aosfatos['title'].astype(str).apply(clean_special_char)

In [173]:
aosfatos['rating'].value_counts()

falso         207
distorcido     33
Name: rating, dtype: int64

In [174]:
aosfatos['rating'] = 0.0

In [175]:
aosfatos['subtitle'] = None

In [176]:
aosfatos['date'] = pd.to_datetime(aosfatos['date'], errors='coerce').dt.date

In [177]:
for column in aosfatos.columns:
    aosfatos[column] = np.where(aosfatos[column].isin(["Nan"]), None, aosfatos[column])

In [178]:
aosfatos['text'] = aosfatos['text'].apply(str.lower)

In [179]:
aosfatos

,url,title,author,date,text,rating,subtitle
0,aosfatos.org,Vídeo em que mar invade avenida no Rio de Jane...,Priscila Pacheco,2022-05-23,não foi gravado recentemente um vídeo que most...,0.0,None
1,aosfatos.org,Avião usado por Lula em 2019 não pertencia à X...,Priscila Pacheco,2022-05-23,é falso que a xp investimentos emprestou um av...,0.0,None
2,aosfatos.org,Lula não apareceu em capa da Forbes como uma...,Marco Faustino,2022-05-20,é falso que uma capa da revista forbes indica ...,0.0,None
3,aosfatos.org,É falso que Eduardo Campos liderava pesquisas ...,Priscila Pacheco,2022-05-20,o ex governador de pernambuco eduardo campos ...,0.0,None
4,aosfatos.org,Vídeo mostra via em Curitiba coberta de graniz...,Marco Faustino,2022-05-19,um vídeo que mostra uma via coberta de gelo em...,0.0,None
...,...,...,...,...,...,...,...
235,aosfatos.org,É falso que Estados Unidos baniram testes RT P...,Marco Faustino,2022-01-06,não é verdade que os estados unidos baniram os...,0.0,None
236,aosfatos.org,É falso que vídeo mostre que barragem tem risc...,Priscila Pacheco,2022-01-05,não é verdade que vídeo compartilhado nas rede...,0.0,None
237,aosfatos.org,Não é verdade que Ivete Sangalo anunciou que d...,Marco Faustino,2022-01-05,é falso que a cantora ivete sangalo anunciou q...,0.0,None
238,aosfatos.org,Vídeo que mostra protesto contra Lula não é re...,Marco Faustino,2022-01-04,não é recente nem foi gravado no recife pe o...,0.0,None


In [180]:
aosfatos = aosfatos.reindex(['title', "subtitle", "text", "tag", "date", "author", "url", "rating"], axis=1)

In [181]:
aosfatos['tag'] = None

In [1806]:
aosfatos

,title,subtitle,text,tag,date,author,url,rating,text_processed
0,Vídeo em que mar invade avenida no Rio de Jane...,None,não foi gravado recentemente um vídeo que most...,None,2022-05-23,Priscila Pacheco,aosfatos.org,0.0,"[nã, grav, recent, víd, mostr, invad, aven, le..."
1,Avião usado por Lula em 2019 não pertencia à X...,None,é falso que a xp investimentos emprestou um av...,None,2022-05-23,Priscila Pacheco,aosfatos.org,0.0,"[fals, xp, invest, emprest, aviã, jat, ex, pre..."
2,Lula não apareceu em capa da Forbes como uma...,None,é falso que uma capa da revista forbes indica ...,None,2022-05-20,Marco Faustino,aosfatos.org,0.0,"[fals, cap, revist, forb, indic, ex, president..."
3,É falso que Eduardo Campos liderava pesquisas ...,None,o ex governador de pernambuco eduardo campos ...,None,2022-05-20,Priscila Pacheco,aosfatos.org,0.0,"[ex, govern, pernambuc, eduard, camp, 1965, 20..."
4,Vídeo mostra via em Curitiba coberta de graniz...,None,um vídeo que mostra uma via coberta de gelo em...,None,2022-05-19,Marco Faustino,aosfatos.org,0.0,"[víd, mostr, via, cobert, gel, curitib, pr, nã..."
...,...,...,...,...,...,...,...,...,...
235,É falso que Estados Unidos baniram testes RT P...,None,não é verdade que os estados unidos baniram os...,None,2022-01-06,Marco Faustino,aosfatos.org,0.0,"[nã, verdad, unid, ban, test, usam, métod, rt,..."
236,É falso que vídeo mostre que barragem tem risc...,None,não é verdade que vídeo compartilhado nas rede...,None,2022-01-05,Priscila Pacheco,aosfatos.org,0.0,"[nã, verdad, víd, compartilh, red, soc, vej, a..."
237,Não é verdade que Ivete Sangalo anunciou que d...,None,é falso que a cantora ivete sangalo anunciou q...,None,2022-01-05,Marco Faustino,aosfatos.org,0.0,"[fals, cantor, ivet, sangal, anunc, deix, bras..."
238,Vídeo que mostra protesto contra Lula não é re...,None,não é recente nem foi gravado no recife pe o...,None,2022-01-04,Marco Faustino,aosfatos.org,0.0,"[nã, recent, grav, recif, víd, mostr, protest,..."


In [182]:
aosfatos.to_csv("aosfatos_cleaned.tsv")

## Cleaning Folha

In [204]:
folha = pd.read_csv("C:\\Users\\franc\\IH-Lab\\BR-FakeNews-Detector\\Data\\folha.csv").drop('subcategory', axis = 1)

In [205]:
folha["link"] = folha["link"].astype(str).apply(clean_website)

In [206]:
def clean_tagfolha(t):
    
    if t == "poder":
        t = "política"
    elif t == "bbc" or t == "mundo" or t == "euronews" or t == "rfi" or t == "dw":
        t = "mundo"
    elif t == "ciencia":
        t = "ciência"
    elif t == "colunas" or t == "tv" or t == "esporte" or t == "musica" or t == "turismo" or t == "tec":
        t = "entretenimento"
    elif t == "equilibrioesaude":
        t = "saúde"
    elif t == "empreendedorsocial" or t == "mercado":
        t = "economia"
    elif t == "saopaulo" or t == "educacao":
        t = "brasil"
    else:
        t = "outros"
    
    return t

folha['category'].value_counts(dropna =False) #saúde, política, entretenimento, brasil, ciência, mundo,

In [207]:
folha['category'] = folha['category'].astype(str).apply(clean_tagfolha)

In [208]:
folha['date'] = pd.to_datetime(folha['date'], errors='coerce').dt.date

In [209]:
folha['date'].isna().sum()

0

In [210]:
folha['rating'] = 1.0

In [211]:
folha['author'] = "Folha de São Paulo"

In [212]:
folha['subtitle'] = None

In [213]:
folha['url'] = folha['link']

In [214]:
folha['tag'] = folha['category']

In [215]:
folha.drop(['link', 'category'], axis =1)

,title,text,date,rating,author,subtitle,url,tag
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,1.0,Folha de São Paulo,None,folha.uol.com.br,política
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,1.0,Folha de São Paulo,None,folha.uol.com.br,outros
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,1.0,Folha de São Paulo,None,folha.uol.com.br,política
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,1.0,Folha de São Paulo,None,folha.uol.com.br,outros
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,1.0,Folha de São Paulo,None,folha.uol.com.br,economia
...,...,...,...,...,...,...,...,...
167048,"Em cenário de crise, tucano Beto Richa assume ...",O tucano Beto Richa tinha tudo para começar se...,2015-01-01,1.0,Folha de São Paulo,None,folha.uol.com.br,política
167049,Filho supera senador Renan Calheiros e assume ...,O economista Renan Filho (PMDB) assume nesta q...,2015-01-01,1.0,Folha de São Paulo,None,folha.uol.com.br,política
167050,"Hoje na TV: Tottenham x Chelsea, Campeonato In...",Destaques da programação desta quinta-feira (1...,2015-01-01,1.0,Folha de São Paulo,None,folha.uol.com.br,entretenimento
167051,Kim Jong-un diz estar aberto a se reunir com p...,"O líder norte-coreano, Kim Jong-un, disse nest...",2015-01-01,1.0,Folha de São Paulo,None,folha.uol.com.br,mundo


In [216]:
folha = folha.reindex(['title', "subtitle", "text", "tag", "date", "author", "url", "rating"], axis=1)

In [217]:
folha['text'] = folha['text'].str.lower()

In [219]:
folha['text'].dropna(inplace = True)

In [ ]:
folha.to_csv("folha_cleaned.tsv")